In [12]:
import os
import matplotlib.pyplot as plt
from nilearn import image as nimg
from nilearn import plotting as nplot
from bids import BIDSLayout
import nibabel as nib
import ants
from ants import resample_image
import re
import shutil

<h3> Brain Mask </h3>

In [8]:
EP = [f.path for f in os.scandir("FullData/EP/RAW")]
HC = [f.path for f in os.scandir("FullData/HC/RAW")]

In [3]:
count = 0
command = []

for i in EP:
    outFile = '"FullData/EP/Filtered/SB'+str(count)+'.nii.gz"'
    outMask = '"FullData/EP/BrainMask/SB'+str(count)+'.nii.gz"'
    command.append('skullstrip/synthstrip-docker -i ' + '"' +i +'"'+ ' -o '+ outFile +' -m '+ outMask)
    count += 1

In [4]:
for i in command:
    print(i)
    os.system(i)

skullstrip/synthstrip-docker -i "FullData/EP/RAW/subject249_T1w.nii.gz" -o "FullData/EP/Filtered/SB0.nii.gz" -m "FullData/EP/BrainMask/SB0.nii.gz"
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: /home/bryan/Projects/SeizTheAttention/FullData/EP/RAW/subject249_T1w.nii.gz
Masked image saved to: /home/bryan/Projects/SeizTheAttention/FullData/EP/Filtered/SB0.nii.gz
Binary brain mask saved to: /home/bryan/Projects/SeizTheAttention/FullData/EP/BrainMask/SB0.nii.gz
If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image.
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann.
NeuroImage 206 (2022), 119474.
Running SynthStrip from Docker
skullstrip/synthstrip-docker -i "FullData/EP/RAW/subject196_T1w.nii.gz" -o "FullData/EP/Filtered/SB1.nii.gz" -m "FullData/EP/BrainMask/SB1.nii.gz"
Configuring model on the CPU
Running SynthStrip model version 1
Input image read fr

In [5]:
count = 0
command = []

for i in HC:
    outFile = '"FullData/HC/Filtered/SB'+str(count)+'.nii.gz"'
    outMask = '"FullData/HC/BrainMask/SB'+str(count)+'.nii.gz"'
    command.append('skullstrip/synthstrip-docker -i ' + '"' +i +'"'+ ' -o '+ outFile +' -m '+ outMask)
    count += 1

In [6]:
for i in command:
    print(i)
    os.system(i)

skullstrip/synthstrip-docker -i "FullData/HC/RAW/subject110_T1w.nii.gz" -o "FullData/HC/Filtered/SB0.nii.gz" -m "FullData/HC/BrainMask/SB0.nii.gz"
Configuring model on the CPU
Running SynthStrip model version 1
Input image read from: /home/bryan/Projects/SeizTheAttention/FullData/HC/RAW/subject110_T1w.nii.gz
Masked image saved to: /home/bryan/Projects/SeizTheAttention/FullData/HC/Filtered/SB0.nii.gz
Binary brain mask saved to: /home/bryan/Projects/SeizTheAttention/FullData/HC/BrainMask/SB0.nii.gz
If you use SynthStrip in your analysis, please cite:
----------------------------------------------------
SynthStrip: Skull-Stripping for Any Brain Image.
A Hoopes, JS Mora, AV Dalca, B Fischl, M Hoffmann.
NeuroImage 206 (2022), 119474.
Running SynthStrip from Docker
skullstrip/synthstrip-docker -i "FullData/HC/RAW/subject38_T1w.nii.gz" -o "FullData/HC/Filtered/SB1.nii.gz" -m "FullData/HC/BrainMask/SB1.nii.gz"
Configuring model on the CPU
Running SynthStrip model version 1
Input image read fro

<h3> Reorient </h3>

In [2]:
import re

In [5]:
Scans = [f.path for f in os.scandir("AllScans")]

In [4]:
count = 0
for i in Scans:
    name = re.search('/(.*)', i)
    print("preprocess " +i+" -o NormScans/" +name.group(1)+" -2n4")
    os.system("preprocess " +i+" -o NormScans/" +name.group(1)+" -2n4 -r 1")
    count += 1
    print(count)

preprocess AllScans/HC411.nii.gz -o NormScans/HC411.nii.gz -2n4
1
preprocess AllScans/EP68.nii.gz -o NormScans/EP68.nii.gz -2n4
2
preprocess AllScans/EP194.nii.gz -o NormScans/EP194.nii.gz -2n4
3
preprocess AllScans/HC364.nii.gz -o NormScans/HC364.nii.gz -2n4
4
preprocess AllScans/EP143.nii.gz -o NormScans/EP143.nii.gz -2n4
5
preprocess AllScans/EP25.nii.gz -o NormScans/EP25.nii.gz -2n4
6
preprocess AllScans/EP124.nii.gz -o NormScans/EP124.nii.gz -2n4
7
preprocess AllScans/HC288.nii.gz -o NormScans/HC288.nii.gz -2n4
8
preprocess AllScans/EP49.nii.gz -o NormScans/EP49.nii.gz -2n4
9
preprocess AllScans/EP152.nii.gz -o NormScans/EP152.nii.gz -2n4
10
preprocess AllScans/EP22.nii.gz -o NormScans/EP22.nii.gz -2n4
11
preprocess AllScans/EP185.nii.gz -o NormScans/EP185.nii.gz -2n4
12
preprocess AllScans/HC274.nii.gz -o NormScans/HC274.nii.gz -2n4
13
preprocess AllScans/HC296.nii.gz -o NormScans/HC296.nii.gz -2n4
14
preprocess AllScans/EP211.nii.gz -o NormScans/EP211.nii.gz -2n4
15
preprocess A

<h3> Voxel Normalization </h3>

In [4]:
Scans = [f.path for f in os.scandir("NormScans")]

In [17]:
count = 0
for i in Scans:
    name = re.search('/(.*)', i)
    t1_img = nib.load(i)
    ants_image = ants.from_nibabel(t1_img)
    ants_image = resample_image(ants_image, (216, 180, 216), use_voxels=True)
    _image = ants_image.to_nibabel()
    nib.save(_image, os.path.join('spaciallyNormal', name.group(1)))
    

In [3]:
Scans = [f.path for f in os.scandir("spaciallyNormal")]

In [4]:
for i in Scans:
    t1_img = nib.load(i)
    t1_data = t1_img.get_fdata()
    print(t1_data.shape)

(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 180, 216)
(216, 18

<h3> after lsq normalization, move all scans to one file </h3>

In [11]:
Scans = [f.path for f in os.scandir("LSQnormal")]

In [14]:
for i in Scans:
    if "lsq" in i:
        name = re.search('/(.*)', i)
        shutil.move(i, "FinalData/"+name.group(1))

In [10]:
t1_data.shape

(216, 180, 216)

In [ ]:
f